In [ ]:
from tkinter import *
from tkinter.ttk import *
from pymongo import *
from bson import ObjectId
import datetime as dt
modelo_factura={
  "_id": {
    "$oid": "60c72b2f9b8c1a001c2e4f06"
  },
  "numero_factura": "",
  "fecha": {
    "$date": ""
  },
  "cliente": {
    "nombre": "",
    "email": "",
    "telefono":"",
    "direccion": {
      "calle": "",
      "ciudad": "",
      "codigo_postal": ""
    },
    "dni": ""
  },
  "items": [
    
  ],
  "total_impuestos": 0,
  "total_factura": 0,
  "estado": ""
}

class FrmDialog:
    def __init__(self,main_form):
        self.root=Toplevel(main_form)
        self.root.geometry("600x400")
        self.root.focus_set()
      
class FrmFacturacion(FrmDialog):
    def __init__(self, main_form):
        super().__init__(main_form)
        # Variables para los Entry (opcional, pero útil para gestión de datos)
        # Cabecera/Factura
        self.var_numero_factura = StringVar()
        self.var_fecha = StringVar()
        self.inicializar_cabecera()
        # Cliente
        self.var_cliente_dni = StringVar()
        self.var_cliente_nombre = StringVar()
        self.var_cliente_email = StringVar()
        self.var_cliente_calle = StringVar()
        self.var_cliente_ciudad = StringVar()
        self.var_cliente_cp = StringVar()
        self.doc_cli=dict()
        # Nuevo Ítem
        self.var_item_nombre = StringVar()
        self.var_item_cantidad = StringVar(value="1")
        self.var_item_precio = StringVar()
        
        # Resumen
        self.var_total_impuestos = StringVar(value="0.00")
        self.var_total_factura = StringVar(value="0.00")

        self.crear_widgets()

    def crear_widgets(self):
        """Método principal para organizar la estructura de la interfaz."""
        
        # --- Configuración de Estilos ---
        style = Style()
        style.configure("TFrame", padding=10)
        style.configure("TLabel", font=('Arial', 10))
        style.configure("TEntry", font=('Arial', 10))
        style.configure("Treeview.Heading", font=('Arial', 10, 'bold'))

        # --- Frames Principales ---
        
        # 1. Cabecera (Información de la factura y Cliente)
        self.frame_cabecera =LabelFrame(self. root, text="Datos de Factura y Cliente", padding="10")
        self.frame_cabecera.pack(padx=10, pady=10, fill="x")

        # 2. Ítems (TreeView y Controles de Adición)
        self.frame_items = LabelFrame(self. root, text="Detalle de Ítems", padding="10")
        self.frame_items.pack(padx=10, pady=10, fill="both", expand=True)

        # 3. Resumen y Botones
        self.frame_resumen = LabelFrame(self. root, text="Resumen de Totales", padding="10")
        self.frame_resumen.pack(padx=10, pady=10, fill="x")

        
        # =================================================================
        # ## 1. Cabecera y Cliente
        # =================================================================
        
        # Sub-frame para Número y Fecha de Factura (Izquierda de la cabecera)
        frame_factura_info =  Frame(self.frame_cabecera)
        frame_factura_info.pack(side="left", padx=10, pady=5, fill="x")
        
        Label(frame_factura_info, text="Nº Factura:").grid(row=0, column=0, sticky="w", padx=5, pady=2)
        Entry(frame_factura_info, textvariable=self.var_numero_factura, width=15).grid(row=0, column=1, sticky="w", padx=5, pady=2)
        
        Label(frame_factura_info, text="Fecha:").grid(row=1, column=0, sticky="w", padx=5, pady=2)
        Entry(frame_factura_info, textvariable=self.var_fecha, width=15, state='readonly').grid(row=1, column=1, sticky="w", padx=5, pady=2) # La fecha suele ser autogenerada o un control de calendario

        # Sub-frame para Datos de Cliente (Derecha de la cabecera, como cabecera)
        frame_cliente_info = Frame(self.frame_cabecera)
        frame_cliente_info.pack(side="right", padx=10, pady=5, fill="x")
        
        # DNI con botón de búsqueda (como se solicitó)
        Label(frame_cliente_info, text="DNI/NIF:").grid(row=0, column=0, sticky="w", padx=5, pady=2)
        entry_dni = Entry(frame_cliente_info, textvariable=self.var_cliente_dni, width=15)
        entry_dni.grid(row=0, column=1, sticky="w", padx=5, pady=2)
        Button(frame_cliente_info, text="Buscar", width=8,command=lambda : self.consultar(self.var_cliente_dni)).grid(row=0, column=2, sticky="w", padx=5, pady=2)
        
        # Datos del cliente recuperados (a modo de cabecera)
        Label(frame_cliente_info, text="Nombre:").grid(row=1, column=0, sticky="w", padx=5, pady=2)
        Entry(frame_cliente_info, textvariable=self.var_cliente_nombre, width=30, state='readonly').grid(row=1, column=1, columnspan=2, sticky="w", padx=5, pady=2)
        
        Label(frame_cliente_info, text="Email:").grid(row=2, column=0, sticky="w", padx=5, pady=2)
        Entry(frame_cliente_info, textvariable=self.var_cliente_email, width=30, state='readonly').grid(row=2, column=1, columnspan=2, sticky="w", padx=5, pady=2)
        
        Label(frame_cliente_info, text="Dirección:").grid(row=3, column=0, sticky="w", padx=5, pady=2)
        Entry(frame_cliente_info, textvariable=self.var_cliente_calle, width=30, state='readonly').grid(row=3, column=1, columnspan=2, sticky="w", padx=5, pady=2)
        
        Label(frame_cliente_info, text="Ciudad/CP:").grid(row=4, column=0, sticky="w", padx=5, pady=2)
        Entry(frame_cliente_info, textvariable=self.var_cliente_ciudad, width=15, state='readonly').grid(row=4, column=1, sticky="w", padx=5, pady=2)
        Entry(frame_cliente_info, textvariable=self.var_cliente_cp, width=10, state='readonly').grid(row=4, column=2, sticky="w", padx=5, pady=2)

        
        # =================================================================
        # ## 2. Ítems
        # =================================================================

        # --- Listview (Treeview) para Ítems Añadidos ---
        
        self.tree =  Treeview(self.frame_items, columns=("Nombre", "Cantidad", "Precio Unitario", "Subtotal"), show="headings", height=10)
        
        # Definición de las columnas
        self.tree.heading("Nombre", text="Nombre Producto")
        self.tree.heading("Cantidad", text="Cant.")
        self.tree.heading("Precio Unitario", text="P. Unitario")
        self.tree.heading("Subtotal", text="Subtotal")
        
        # Ajuste de anchos
        self.tree.column("Nombre", width=300, anchor="w")
        self.tree.column("Cantidad", width=70, anchor="center")
        self.tree.column("Precio Unitario", width=100, anchor="e")
        self.tree.column("Subtotal", width=100, anchor="e")
        
        self.tree.pack(fill="x", pady=5)
        
        # Barra de desplazamiento (Scrollbar)
        scrollbar = Scrollbar(self.frame_items, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        # Esto es más completo para `pack`:
        scrollbar.pack(side="right", fill="y")
        self.tree.pack(side="left", fill="both", expand=True)
        # Re-pack para asegurar que ambos estén bien posicionados
        scrollbar.pack_forget() 
        self.tree.pack_forget()
        self.tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        self.tree.configure()
        # --- Controles para Nuevo Ítem (Debajo de la Listview) ---
        
        frame_nuevo_item = Frame(self.frame_items, padding="5")
        frame_nuevo_item.pack(fill="x", pady=(10, 0))

        # Línea Superior: Etiquetas (Labels)
        Label(frame_nuevo_item, text="Nombre Producto:").grid(row=0, column=0, padx=5, sticky="w")
        Label(frame_nuevo_item, text="Cantidad:").grid(row=0, column=1, padx=5, sticky="w")
        Label(frame_nuevo_item, text="Precio Unitario:").grid(row=0, column=2, padx=5, sticky="w")
        Label(frame_nuevo_item, text="").grid(row=0, column=3, padx=5, sticky="w") # Espacio para el botón

        # Línea Inferior: Entradas (Entries) y Botón
        Entry(frame_nuevo_item, textvariable=self.var_item_nombre, width=40).grid(row=1, column=0, padx=5, pady=2, sticky="ew")
        Entry(frame_nuevo_item, textvariable=self.var_item_cantidad, width=10).grid(row=1, column=1, padx=5, pady=2, sticky="ew")
        Entry(frame_nuevo_item, textvariable=self.var_item_precio, width=15).grid(row=1, column=2, padx=5, pady=2, sticky="ew")
        
        # Botón para añadir
        Button(frame_nuevo_item, text="Añadir Ítem", width=15).grid(row=1, column=3, padx=5, pady=2, sticky="w")

        # Configurar la expansión para la columna 0 (Nombre Producto)
        frame_nuevo_item.grid_columnconfigure(0, weight=1)

        
        # =================================================================
        # ## 3. Resumen y Botones
        # =================================================================

        # Sub-frame para Totales (Derecha del resumen, como se pide en la parte inferior)
        frame_totales = Frame(self.frame_resumen)
        frame_totales.pack(side="right", fill="x", padx=10, pady=5)
        
        Label(frame_totales, text="Total Impuestos:").grid(row=0, column=0, sticky="w", padx=5, pady=2)
        Entry(frame_totales, textvariable=self.var_total_impuestos, width=15, justify="right", state='readonly').grid(row=0, column=1, sticky="e", padx=5, pady=2)
        
        Label(frame_totales, text="TOTAL FACTURA:", font=('Arial', 11, 'bold')).grid(row=1, column=0, sticky="w", padx=5, pady=5)
        Entry(frame_totales, textvariable=self.var_total_factura, width=15, justify="right", font=('Arial', 11, 'bold'), state='readonly').grid(row=1, column=1, sticky="e", padx=5, pady=5)
        
        # Sub-frame para Botones de Acción (Izquierda del resumen)
        frame_acciones = Frame(self.frame_resumen)
        frame_acciones.pack(side="left", fill="x", padx=10, pady=5)
        
        Button(frame_acciones, text="Guardar Factura", style="Accent.TButton", width=18).grid(row=0, column=0, padx=10)
        Button(frame_acciones, text="Limpiar", width=15).grid(row=0, column=1, padx=10)
        
        # Configurar el estilo del botón "Guardar"
        style.configure("Accent.TButton", foreground="white", background="#4CAF50", font=('Arial', 10, 'bold'))


    ##funciones generales
    def inicializar_cabecera(self):
        self.var_numero_factura.set(self.getMax())
        self.var_fecha.set(str(dt.datetime.now()))
        modelo_factura["numero_factura"]=self.var_numero_factura.get()
        modelo_factura["fecha"]={"$date":self.var_fecha.get()}
    def inicializar_cliente(self,doc_cli):
        self.var_cliente_calle.set(self.doc_cli["direccion"]["calle"])
        self.var_cliente_ciudad.set(self.doc_cli["direccion"]["ciudad"])
        self.var_cliente_cp.set(self.doc_cli["direccion"]["codigo_postal"])
        self.var_cliente_dni.set(self.doc_cli["dni"])
        self.var_cliente_nombre.set(self.doc_cli["nombre"])
        self.var_cliente_email.set(self.doc_cli["email"])
        modelo_factura["cliente"]["direccion"]=doc_cli["direccion"]
        modelo_factura["cliente"]["nombre"]
    def consultar(self,dni):
        cliente=MongoClient("mongodb://127.0.0.1:27017")
        db=cliente.get_database("FacturacionDb")
        
        self.doc_cli=db.clientes.find_one({"dni":dni.get()},{"cliente_id":1,"nombre":1,"email":1,"direccion":1,"dni":1})
        self.inicializar_cliente(self.doc_cli)

    def getMax(self):
        cliente=MongoClient("mongodb://127.0.0.1:27017")
        db=cliente.get_database("FacturacionDb")
        actual_y=str(dt.datetime.now().year)
        print(actual_y)
        cur=db.facturas.find({"numero_factura":{"$regex":f"INV-{actual_y}"}},{"numero_factura":1}).sort({"numero_factura":-1}).limit(1)
        doc=next(cur,None)
        partes=doc["numero_factura"].split(sep="-")
        partes[2]=str(int(partes[2])+1) 
        return "-".join(partes)            

class FrmMantenimiento(FrmDialog):
    def __init__(self, main_form):
        super().__init__(main_form)

class FrmMain:
    def __init__(self):
        self.root=Tk()
        self.root.geometry("800x600")
        mnu_bar=Menu()
        self.root.config(menu=mnu_bar)
        mnu_mantenimiento=Menu(title="Mantenimiento",tearoff=0)
        mnu_mantenimiento.add_cascade(label="Mantenimiento Clientes",command=self.on_mnu_clientes)
        mnu_mantenimiento.add_cascade(label="Mantenimiento Productos")
        mnu_bar.add_cascade(menu=mnu_mantenimiento,label="Mantenimiento")
    
    def on_mnu_clientes(self):
        FrmFacturacion(self.root)

    def mostrar(self):
        self.root.mainloop()

FrmMain().mostrar()

2025


In [38]:
cadena="INV-2025-001"
partes=cadena.split(sep="-")
num=int(partes[2])
print(num+1)
partes[2]=str(num+1)
nuevo=""
print(partes)
print("-".join(partes))

2
['INV', '2025', '2']
INV-2025-2
